<a href="https://colab.research.google.com/github/rg-smith/remote-sensing-hydro-2026/blob/main/lectures/lecture2-raster-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this demonstration, we use gridded climate data from the PRISM dataset to analyze how precipitation varies over the Corcoran Clay region in California’s Central Valley.

We will visualize annual precipitation as raster maps, clip the data to the corcroan clay, and compute yearly average rainfall over the region. This shows how geospatial tools can be used to summarize large climate datasets.

In [ ]:
# Install required libraries (only needed the first time you run this notebook in Colab)
!pip install -q geemap earthengine-api geopandas



In [ ]:
# Import libraries for working with Earth Engine, maps, and data tables
import ee                  # Google Earth Engine Python API
import geemap              # Interactive mapping with Earth Engine
import geopandas as gpd    # For handling vector (shapefile/GeoJSON) data
import pandas as pd        # For working with tables and time series
import matplotlib.pyplot as plt  # For plotting graphs


In [ ]:
# Authenticate with Google Earth Engine.
# This will open a link where you sign in with your Google account
# and paste the authorization code back here.
ee.Authenticate()




In [ ]:
# Initialize the Earth Engine connection (connects Python to your EE account)
ee.Initialize(project='please fill in from google earth engine')


In [ ]:
# Load the PRISM monthly climate dataset from Earth Engine.
# Each image represents total precipitation for one month (mm/month).
prism = ee.ImageCollection("OREGONSTATE/PRISM/ANm")

# Print how many monthly images are in the dataset (sanity check)
print(prism.size().getInfo())


In [ ]:
# Upload local files from your computer (for example, Corcoran Clay shapefile or GeoJSON)
# After running this cell, choose the file(s) from your laptop
from google.colab import files
uploaded = files.upload()

# Show the names of the uploaded files
print("Uploaded files:", list(uploaded.keys()))


In [ ]:
# Read the uploaded Corcoran Clay shapefile into GeoPandas
# (make sure all shapefile parts were uploaded: .shp, .shx, .dbf, .prj)
gdf = gpd.read_file("corcoran_clay_extent.shp")

# Convert the GeoPandas layer to an Earth Engine FeatureCollection
# so we can use it for clipping and spatial analysis with raster data
corcoran = geemap.geopandas_to_ee(gdf)


In [ ]:
# Create an interactive map centered over California’s Central Valley
Map = geemap.Map(center=[36.5, -119.5], zoom=6)
Map.add_basemap("TERRAIN")

# -------------------------------------------------------
# Calculate ANNUAL TOTAL PRISM precipitation for 2017
# PRISM data are monthly totals (mm/month), so summing
# all 12 months gives total precipitation for the year (mm/year)
# -------------------------------------------------------
img = prism.select("ppt").filterDate("2017-01-01", "2018-01-01").sum()

# -------------------------------------------------------
# Automatically compute min and max precipitation values
# inside the Corcoran Clay region (for proper color scaling)
# -------------------------------------------------------
stats = img.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=corcoran.geometry(),
    scale=4000,
    maxPixels=1e13
)

ppt_min = stats.get("ppt_min").getInfo()
ppt_max = stats.get("ppt_max").getInfo()

# -------------------------------------------------------
# Visualization settings (auto-scaled to data range)
# Red = drier areas, Blue = wetter areas
# -------------------------------------------------------
vis = {
    "min": ppt_min,
    "max": ppt_max,
    "palette": ["red", "orange", "yellow", "cyan", "blue"]
}

# Add the precipitation raster clipped to Corcoran Clay
Map.addLayer(img.clip(corcoran), vis, "PRISM Annual Precip (2017)")

# Add Corcoran Clay boundary outline
Map.addLayer(
    corcoran.style(**{"color": "black", "fillColor": "00000000", "width": 2}),
    {},
    "Corcoran Clay"
)
# Add a tool so clicking on the map shows pixel values
Map.add_inspector()

Map #visualize the map :)



In [ ]:
# Select only the precipitation band (ppt) from the PRISM dataset
# Units: millimeters per month (mm/month)
prism_ppt = prism.select("ppt")

# -------------------------------------------------------
# Function to calculate ANNUAL TOTAL precipitation
# and then average it over the Corcoran Clay region
# -------------------------------------------------------
def annual_total_over_corcoran(year):

    # Define start and end dates for the year
    start = ee.Date.fromYMD(year, 1, 1)
    end   = ee.Date.fromYMD(year, 12, 31)

    # Sum monthly precipitation to get total rainfall for the year (mm/year)
    annual_img = prism_ppt.filterDate(start, end).sum()

    # Compute spatial mean precipitation over Corcoran Clay
    mean_dict = annual_img.reduceRegion(
        reducer=ee.Reducer.mean(),          # average over space
        geometry=corcoran.geometry(),       # only inside Corcoran Clay
        scale=4000,                         # PRISM resolution ~4 km
        maxPixels=1e13
    )

    # Store results as a feature with year and precipitation value
    return ee.Feature(None, {
        "year": year,
        "annual_ppt_mm": mean_dict.get("ppt")
    })

# -------------------------------------------------------
# Run the function for every year from 2000 to 2025
# and store results in a FeatureCollection
# -------------------------------------------------------
years = list(range(2000, 2026))
features = [annual_total_over_corcoran(y) for y in years]
fc = ee.FeatureCollection(features)


In [ ]:
# -------------------------------------------------------
# Bring Earth Engine results into Python for analysis
# -------------------------------------------------------

# Download the FeatureCollection from Earth Engine as a Python dictionary
data = fc.getInfo()["features"]

# Extract only the attribute values (year and precipitation)
rows = []
for f in data:
    props = f["properties"]
    rows.append(props)

# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(rows)

# Sort by year and reset the index for clean plotting
df = df.sort_values("year").reset_index(drop=True)

# Display the table
df


In [ ]:
# -------------------------------------------------------
# Plot time series of annual precipitation over Corcoran Clay
# -------------------------------------------------------

# Compute long-term average precipitation across all years
mean_val = df["annual_ppt_mm"].mean()

# Create the figure
plt.figure(figsize=(11,4))

# Plot annual precipitation values
plt.plot(df["year"], df["annual_ppt_mm"], marker="o", linewidth=2)

# Add dashed line showing long-term mean
plt.axhline(mean_val, color="k", linestyle="--", label="Long-term mean")

# Add labels and title
plt.xlabel("Year")
plt.ylabel("Annual Precipitation (mm/year)")
plt.title("PRISM Annual Precipitation over Corcoran Clay (2000–2025)")

# Add legend and grid
plt.legend()
plt.grid(True)

# Display the plot
plt.show()
